# Learn the Basics of ∇-Prox

This tutorial introduces you to a completed proximal optimization workflow implemented in ∇-Prox. For simplicity, we will use image deconvolution problem as an example to demonstrate the functionalities of ∇-Prox.

The goal of image deconvolution is to reconstruct the clear image $x$ from the blurred observation $y$ that is obtained by convolving $x$ by the point spread function (PSF) as 
$$
y = D(x, \text{PSF})
$$
where $D$ denotes convolution operation.

To reconstruct target image x from noise-contaminated measurements y, we minimize the sum of a data-fidelity $|| D(x) - y ||^2_2$ and regularizer term $r$ as,
$$
 \mathop{\mathrm{min}}_{x \in \mathbb{R}^n} ~ || D(x) - y ||^2_2 + r (x ; \, \theta_r).
$$

We consider a hybrid regularizer including (1) an implicit plug-and-play prior $g(x; \theta_r)$ parameterized by $\theta_r$ and (2) non-negative constraint of image. 
$$
r(x; \theta_r) = \lambda g(x; \theta_r) + I_{[0,\infty)},
$$


Note: In order to run the following tutorial, please install the requirements following the [Installation tutorial]()

### Import libraries

In the begining, we import all the necessary libraries. 

In [ ]:
from dprox import *
from dprox.utils import *
from dprox.utils.examples import *
import matplotlib.pyplot as plt
plt.rc('text', usetex=True)

### Prepare Data

Then, let's generate some sample data to play with.

In [ ]:
img = sample('face')
psf = point_spread_function(ksize=15, sigma=5)
y = blurring(img, psf)
print(img.shape, y.shape)
imshow(img, y, titles=[r'ground truth $x$', r'blurry measurement $y$'])

### Representing Optimization Problem in ∇-Prox

Recall our goal is to reconstruct the clear image given the blurry observation, by solving an optimization problem as 
$$
 \mathop{\mathrm{min}}_{x \in \mathbb{R}^n} ~ || D(x) - y ||^2_2 + \lambda g(x; \theta_r) + I_{[0,\infty)}.
$$

We can write down this problem in ∇-Prox **with a very simple syntax following the math.**

In [ ]:
x = Variable()
data_term = sum_squares(conv(x, psf) - y)
prior_term = deep_prior(x, 'ffdnet_color')  # using a simple FFDNet denoiser as a deep plug-and-play prior 
reg_term = nonneg(x)
objective = data_term + prior_term + reg_term
p = Problem(objective)

### Basic Problem Solving

Solving the problem can be easily achieved by calling the `solve` method with the desired algorithms, e.g., `ADMM` in this case.

You could also try other algorithms, the compatible algorithms and reference performance are listed as below, 

| Key     | Method                                     | Expected PSNR |
|---------|--------------------------------------------|---------------|
| `admm`  | Alternative Direction Method of Multiplier | 31.94         |
| `hqs`   | Half Qudratic Splitting                    | 31.65         |
| `pc`    | Pock Chambolle                             | 29.66         |
| `ladmm` | Linearized ADMM                            | 31.95         |

In [ ]:
out = p.solve(method='admm', x0=y, pbar=True)
imshow(out, titles=[r'reconstruction $\hat{x}$ ' + f'(PSNR: {psnr(out, img):.3f})'])

In many cases,  you need to manually tune the hyperparameters of the algorithm to achieve better performance. For example, let's consider a slightly harder noisy deconvolution problem as
$$
y = D(x, \text{PSF}) + \epsilon
$$
where $\epsilon$ denotes a small amount of Gaussian noise with intensity $\sigma = \frac{5}{255}$.

In [ ]:
y = blurring(img, psf) + np.random.randn(*img.shape).astype('float32') * 5/255.0
imshow(img, y, titles=[r'ground truth $x$', r'blurry and noisy measurement $y$'])

Again, let's solve the problem with ∇-Prox. We can see that default parameters **diverge** and fail to solve the problem.

In [ ]:
x = Variable()
data_term = sum_squares(conv(x, psf) - y)
prior_term = deep_prior(x, 'ffdnet_color')
reg_term = nonneg(x)
objective = data_term + prior_term + reg_term
p = Problem(objective)
out = p.solve(method='admm', x0=y, pbar=True)
imshow(out, titles=[r'reconstruction $\hat{x}_1$ ' + f'(PSNR: {psnr(out, img):.3f})'])

To fix it, we have to manually tune the algorithm parameters. In ∇-Prox, this can be achieved by passing extra keyword arguments to `solve`, e.g., 

In [ ]:
out = p.solve(method='admm', x0=y, rhos=0.004, lams=0.02, max_iter=24, pbar=True)
imshow(out, titles=[r'reconstruction $\hat{x}_2$ ' + f'(PSNR: {psnr(out, img):.3f})'])

We should note that the manual parameter tuning is a tedious process. Different choices of parameters may affect the performance significantly. For example, by changing `rhos` to 0.1, the PSNR drops by 1.3 dB.

In [ ]:
out = p.solve(method='admm', x0=y, rhos=0.1, lams=0.02, max_iter=24, pbar=True)
imshow(out, titles=[r'reconstruction $\hat{x}_3$ ' + f'(PSNR: {psnr(out, img):.3f})'])

In many cases, the optimal choice may vary for different inputs. Moreover, for real-world applications, we won't have ground-truth to evaluate the performance of different parameters for different inputs, which makes parameter tuning even harder.

To address it, ∇-Prox incorporates the automatic parameter scheduler that can be learned with reinforcement learning. Please refer to the [tutorial on automatic parameter scheduler]() for more details.